In [1]:
def loadlist(filename):
    with open(filename, 'r') as f:
        lines = []
        for line in f:
            lines.append(line[:-1])
    return(lines)

In [2]:
linky = loadlist("www.filmovyprehled.cz_all.txt")

In [3]:
linky[0:10]

['https://www.filmovyprehled.cz/en/person/86426/michal-vican',
 'https://www.filmovyprehled.cz/cs/person/84040/boris-krajny',
 'https://www.filmovyprehled.cz/cs/person/27613/jiri-hejl',
 'https://www.filmovyprehled.cz/cs/kalendar/detail/vsetinsky-filmovy-maraton',
 'https://www.filmovyprehled.cz/cs/film/197143/krehka-identita',
 'https://www.filmovyprehled.cz/cs/person/92640/mirage-production-service',
 'https://www.filmovyprehled.cz/en/person/5210/tamir-ment',
 'https://www.filmovyprehled.cz/en/person/24023/sophie-animation-co-ltd',
 'https://www.filmovyprehled.cz/cs/person/30140/alos-neuman',
 'https://www.filmovyprehled.cz/cs/person/76111/lukas-ujcik']

In [4]:
linky_filmy = [x for x in linky if "/cs/film/" in x]

In [5]:
linky_filmy[0:10]

['https://www.filmovyprehled.cz/cs/film/197143/krehka-identita',
 'https://www.filmovyprehled.cz/cs/film/402101/famu-v-kine-01',
 'https://www.filmovyprehled.cz/cs/film/102314/zimni-vila',
 'https://www.filmovyprehled.cz/cs/film/401931/barbora-polakova-krosna',
 'https://www.filmovyprehled.cz/cs/film/395473/jeji-pastorkyna',
 'https://www.filmovyprehled.cz/cs/film/397657/zacatek-dlouheho-podzimu',
 'https://www.filmovyprehled.cz/cs/film/397398/vrak',
 'https://www.filmovyprehled.cz/cs/film/395884/hvezda-z-posledni-stace',
 'https://www.filmovyprehled.cz/cs/film/397649/uzavreny-okruh',
 'https://www.filmovyprehled.cz/cs/film/396286/stenata']

## Začínáme scrapovat

In [6]:
import requests
from bs4 import BeautifulSoup

### Nejdřív zjistíme, co všechno budeme scrapovat

In [8]:
filmy_benchmark = ["kolja",
                   "pelisky",
                   "postriziny",
                   "lazarova",
                   "spalovac",
                   "dostavenicko-ve-mlynici",
                   "prazska-5",
                   "bersidejsi"]
linky_benchmark = []
for l in linky_filmy:
    for f in filmy_benchmark:
        if f in l:
            linky_benchmark.append(l)
linky_benchmark

['https://www.filmovyprehled.cz/cs/film/396753/marketa-lazarova',
 'https://www.filmovyprehled.cz/cs/film/9013/kolja',
 'https://www.filmovyprehled.cz/cs/film/396780/spalovac-mrtvol',
 'https://www.filmovyprehled.cz/cs/film/397282/postriziny',
 'https://www.filmovyprehled.cz/cs/film/397584/bersidejsi',
 'https://www.filmovyprehled.cz/cs/film/395109/dostavenicko-ve-mlynici',
 'https://www.filmovyprehled.cz/cs/film/397598/prazska-5',
 'https://www.filmovyprehled.cz/cs/film/37635/pelisky']

In [9]:
def jake_jsou_detaily(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    item_details = soup.find_all(class_='item-details')
    h3_texts = []
    for item_detail in item_details:
        h3_tags = item_detail.find_all('h3')
        for h3_tag in h3_tags:
            h3_texts.append(h3_tag.get_text())
    return(h3_texts)

In [10]:
detaily = []
for l in linky_benchmark:
    nove_detaily = jake_jsou_detaily(l)
    for n in nove_detaily:
        if n not in detaily:
            detaily.append(n)
detaily

['Země původu',
 'Copyright',
 'Rok výroby',
 'Premiéra',
 'Minutáž',
 'Režie',
 'Kategorie',
 'Žánr',
 'Typologie',
 'Originální název',
 'Český název',
 'Anglický název',
 'povídkový film',
 'Povídky',
 'Souběžný název',
 'Souběžný anglický název']

In [15]:
def jake_jsou_sekce(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    item_details = soup.find_all(class_='db-film-section')
    h2_texts = []
    for item_detail in item_details:
        h2_tags = item_detail.find_all('h2')
        for h2_tag in h2_tags:
            h2_texts.append(h2_tag.get_text())
    return(h2_texts)

In [16]:
jake_jsou_sekce("https://www.filmovyprehled.cz/cs/film/397598/prazska-5")

['Anotace',
 'Obsah',
 'Film online',
 'Galerie',
 'Hrají',
 'Štáb a tvůrci',
 'Produkční údaje',
 'Technické údaje',
 'Ocenění',
 'Knihovna']

In [21]:
def jake_jsou_kredity(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    item_details = soup.find_all(class_='db-film-credits-section')
    h3_texts = []
    for item_detail in item_details:
        if "Štáb a tvůrci" in item_detail.text:
            h3_tags = item_detail.find_all('h3')
            for h3_tag in h3_tags:
                h3_texts.append(h3_tag.get_text())
    return(h3_texts)

In [23]:
kredity = []
for l in linky_benchmark:
    nove_kredity = jake_jsou_kredity(l)
    for n in nove_kredity:
        if n not in kredity:
            kredity.append(n)
kredity

['Režie',
 'Pomocná režie',
 'Asistent režie',
 'Skript',
 'Původní filmový námět',
 'Předloha',
 'Scénář',
 'Technický scénář',
 'Dramaturg',
 'Kamera',
 'Druhá kamera',
 'Asistent kamery',
 'Architekt',
 'Assistent architekta',
 'Výtvarník',
 'Výprava',
 'Návrhy kostýmů',
 'Kostýmy',
 'Masky',
 'Střih',
 'Asistent střihu',
 'Zvuk',
 'Triky',
 'Návrhy titulků',
 'Vedoucí výroby',
 'Zástupce vedoucího výroby',
 'Asistent vedoucího výroby',
 'Odborný poradce',
 'Spolupráce',
 'Výkonná produkce',
 'Producent',
 'Koproducent',
 'Zvláštní efekty',
 'Koordinátor kaskadérů',
 'Kaskadéři']

## Scrapování filmů

In [54]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [56]:
def scrape_film(url):

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    atributy = {}
    
    ## Detaily – země původu, copyright…
    
    item_details = soup.find_all(class_='item-details')
    divs_text = []

    for item_detail in item_details:
        divs = item_detail.find_all('div')
        for div in divs:
            if div.find_all("span"):
                prvni = div.find("h3").text
                spany = div.find_all("span")
                spany2 = []
                for s in spany:
                    spany2.append(s.text)
                druhy = ", ".join(spany2)
                divs_text.append(prvni + " " + druhy)
            else:
                divs_text.append(div.text)
    
    for d in detaily:
        for t in divs_text:
            if d in t:
                t = t.replace(d,"")
                atributy[d] = t.strip()
    
    ## Credits
    
    item_details = soup.find_all(class_='db-film-credits-section')
    divs_text = []

    for item_detail in item_details:
        if "Štáb a tvůrci" in item_detail.text:
            divs = item_detail.find_all('div')
            for div in divs:
                if div.find_all("a"):
                    prvni = div.find("h3").text
                    odkazy = div.find_all("a")
                    odkazy2 = []
                    for a in odkazy:
                        id = id_persony(a["href"])
                        odkazy2.append(f"{a.text} ({id})")
                        druhy = ", ".join(odkazy2)
                        divs_text.append(prvni + " " + druhy)
                else:
                    divs_text.append(div.text)
    
    for k in kredity:
        for d in divs_text:
            if k in d:
                d = d.replace(k,"")
                atributy[k.strip()] = d.strip()
    
    return(atributy)

TODO: DOPLNIT ID

In [57]:
scrape_film("https://www.filmovyprehled.cz/cs/film/397584/bersidejsi")

{'Země původu': 'Československo',
 'Copyright': '1988',
 'Rok výroby': '1987—1988',
 'Minutáž': '13 min',
 'Režie': 'Tomáš Vorel (106)',
 'Kategorie': 'povídka',
 'Žánr': 'podobenství, povídka',
 'Typologie': 'hraný, distribuční, krátkometrážní',
 'Originální název': 'Bersidejsi',
 'Český název': 'Bersidejsi',
 'Anglický název': 'Take-and-Enjoy',
 'povídkový film': 'Pražská 5',
 'Povídky': 'Směr Karlštejn, Bersidejsi, Oldův večírek, Barvy, Na brigádě!',
 'Pomocná režie': 'Věra Pištěková (126010), Dobroslav Zborník (5333)',
 'Skript': 'Dana Šomková (63128)',
 'Původní filmový námět': 'Čestmír Suška (10946)',
 'Scénář': 'Čestmír Suška (10946)',
 'Technický scénář': 'Tomáš Vorel (106), Antonín Weiser (1729), Čestmír Suška (10946)',
 'Dramaturg': 'Tomáš Tintěra (7386), Libuše Hofmanová (126146)',
 'Kamera': 'Antonín Weiser (1729)',
 'Druhá kamera': 'Jan Hanzal (14218), Martin Duba (529)',
 'Asistent kamery': 'Jan Carda (7004)',
 'Architekt': 'Robert Vaněk (78168)',
 'Assistent architekta':